# Getting all bill sponsors from LegiScan

- [ ]  Using `anti-lgbtq-bills-ids.csv` , get bill IDs
- [ ]  Use pylegiscan get_bill to get bill info
- [ ]  Get sponsor for each bill and add to list

## Imports

In [3]:
import io
import json
import csv
import pandas as pd
import numpy as np

[Getting an absolute path from an interactive shell](https://bobbyhadz.com/blog/python-nameerror-name-file-is-not-defined)

## pylegiscan

To talk to LegiScan's API, we're borrowing some code from [pylegiscan](https://github.com/poliquin/pylegiscan). Since it isn't a package you can install with `pip`, it wound up being easier for distribution to just cut and paste it here.

In [4]:
# Taken from https://github.com/poliquin/pylegiscan/blob/master/pylegiscan/legiscan.py

import os
import json
import requests
from urllib.parse import urlencode
from urllib.parse import quote_plus

# current aggregate status of bill
BILL_STATUS = {1: "Introduced",
               2: "Engrossed",
               3: "Enrolled",
               4: "Passed",
               5: "Vetoed",
               6: "Failed/Dead"}

# significant steps in bill progress.
BILL_PROGRESS = {1: "Introduced",
                 2: "Engrossed",
                 3: "Enrolled",
                 4: "Passed",
                 5: "Vetoed",
                 6: "Failed/Dead",
                 7: "Veto Override",
                 8: "Chapter/Act/Statute",
                 9: "Committee Referral",
                10: "Committee Report Pass",
                11: "Committee Report DNP"}


"""
Interact with LegiScan API.

"""

# a helpful list of valid legiscan state abbreviations (no Puerto Rico)
STATES = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga',
          'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma', 'md', 'me',
          'mi', 'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj', 'nm',
          'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx',
          'ut', 'va', 'vt', 'wa', 'wi', 'wv', 'wy']

class LegiScanError(Exception):
    pass

class LegiScan(object):
    BASE_URL = 'http://api.legiscan.com/?key={0}&op={1}&{2}'

    def __init__(self, apikey=None):
        """LegiScan API.  State parameters should always be passed as
           USPS abbreviations.  Bill numbers and abbreviations are case
           insensitive.  Register for API at http://legiscan.com/legiscan
        """
        # see if API key available as environment variable
        if apikey is None:
            apikey = config.LEGISCAN_API_KEY
        self.key = apikey.strip()

    def _url(self, operation, params=None):
        """Build a URL for querying the API."""
        if not isinstance(params, str) and params is not None:
            params = urlencode(params)
        elif params is None:
            params = ''
        return self.BASE_URL.format(self.key, operation, params)

    def _get(self, url):
        """Get and parse JSON from API for a url."""
        req = requests.get(url)
        if not req.ok:
            raise LegiScanError('Request returned {0}: {1}'\
                    .format(req.status_code, url))
        data = json.loads(req.content)
        if data['status'] == "ERROR":
            raise LegiScanError(data['alert']['message'])
        return data

    def get_session_list(self, state):
        """Get list of available sessions for a state."""
        url = self._url('getSessionList', {'state': state})
        data = self._get(url)
        return data['sessions']

    def get_dataset_list(self, state=None, year=None):
        """Get list of available datasets, with optional state and year filtering.
        """
        if state is not None:
            url = self._url('getDatasetList', {'state': state})
        elif year is not None:
            url = self._url('getDatasetList', {'year': year})
        else:
            url = self._url('getDatasetList')
        data = self._get(url)
        # return a list of the bills
        return data['datasetlist']

    def get_dataset(self, id, access_key):
        """Get list of available datasets, with optional state and year filtering.
        """
        url = self._url('getDataset', {'id': id, 'access_key': access_key})
        data = self._get(url)
        # return a list of the bills
        return data['dataset']
      
    def get_master_list(self, state=None, session_id=None):
        """Get list of bills for the current session in a state or for
           a given session identifier.
        """
        if state is not None:
            url = self._url('getMasterList', {'state': state})
        elif session_id is not None:
            url = self._url('getMasterList', {'id': session_id})
        else:
            raise ValueError('Must specify session identifier or state.')
        data = self._get(url)
        # return a list of the bills
        return [data['masterlist'][i] for i in data['masterlist']]

    def get_bill(self, bill_id=None, state=None, bill_number=None):
        """Get primary bill detail information including sponsors, committee
           references, full history, bill text, and roll call information.

           This function expects either a bill identifier or a state and bill
           number combination.  The bill identifier is preferred, and required
           for fetching bills from prior sessions.
        """
        if bill_id is not None:
            url = self._url('getBill', {'id': bill_id})
        elif state is not None and bill_number is not None:
            url = self._url('getBill', {'state': state, 'bill': bill_number})
        else:
            raise ValueError('Must specify bill_id or state and bill_number.')
        return self._get(url)['bill']

    def get_bill_text(self, doc_id):
        """Get bill text, including date, draft revision information, and
           MIME type.  Bill text is base64 encoded to allow for PDF and Word
           data transfers.
        """
        url = self._url('getBillText', {'id': doc_id})
        return self._get(url)['text']

    def get_amendment(self, amendment_id):
        """Get amendment text including date, adoption status, MIME type, and
           title/description information.  The amendment text is base64 encoded
           to allow for PDF and Word data transfer.
        """
        url = self._url('getAmendment', {'id': amendment_id})
        return self._get(url)['amendment']

    def get_supplement(self, supplement_id):
        """Get supplement text including type of supplement, date, MIME type
           and text/description information.  Supplement text is base64 encoded
           to allow for PDF and Word data transfer.
        """
        url = self._url('getSupplement', {'id': supplement_id})
        return self._get(url)['supplement']

    def get_roll_call(self, roll_call_id):
        """Roll call detail for individual votes and summary information."""
        data = self._get(self._url('getRollcall', {'id': roll_call_id}))
        return data['roll_call']

    def get_sponsor(self, people_id):
        """Sponsor information including name, role, and a followthemoney.org
           person identifier.
        """
        url = self._url('getSponsor', {'id': people_id})
        return self._get(url)['person']

    def search(self, state, bill_number=None, query=None, year=2, page=1):
        """Get a page of results for a search against the LegiScan full text
           engine; returns a paginated result set.

           Specify a bill number or a query string.  Year can be an exact year
           or a number between 1 and 4, inclusive.  These integers have the
           following meanings:
               1 = all years
               2 = current year, the default
               3 = recent years
               4 = prior years
           Page is the result set page number to return.
        """
        if bill_number is not None:
            params = {'state': state, 'bill': bill_number}
        elif query is not None:
            params = {'state': state, 'query': query,
                      'year': year, 'page': page}
        else:
            raise ValueError('Must specify bill_number or query')
        data = self._get(self._url('search', params))['searchresult']
        # return a summary of the search and the results as a dictionary
        summary = data.pop('summary')
        results = {'summary': summary, 'results': [data[i] for i in data]}
        return results

    def __str__(self):
        return '<LegiScan API {0}>'.format(self.key)

    def __repr__(self):
        return str(self)

# Connect to LegiScan

Using pylegiscan, you just pass your API key to `LegiScan` and you're good to go. I set up an environment variable for mine, but you can also just paste yours at `OR_PUT_YOUR_API_KEY_HERE`.

In [5]:
import config

api_key = config.LEGISCAN_API_KEY
legis = LegiScan(api_key)

If you wanted to search for bills based on state or text, that's easy to do.

# Read in my anti-trans bills csv

In [6]:
# read in my csv as a dataframe
df = pd.read_csv('anti-lgbtq-bills-ids.csv', usecols=['State','Abbreviation','Number','URL','Bill ID'])

In [7]:
# check the df and columns
df.head()

,State,Abbreviation,Number,URL,Bill ID
0,Alaska,AK,HB27,https://legiscan.com/AK/bill/HB27/2023,1646385
1,Arizona,AZ,HB2312,https://legiscan.com/AZ/bill/HB2312/2023,1657170
2,Arizona,AZ,HB2711,https://legiscan.com/AZ/bill/HB2711/2023,1696919
3,Arizona,AZ,SB1001,https://legiscan.com/AZ/bill/SB1001/2023,1639597
4,Arizona,AZ,SB1026,https://legiscan.com/AZ/bill/SB1026/2023,1644653


In [8]:
# make a new column in the dataframe called Sponsors
df['Sponsors'] = ''

---
# Get sponsors for one bill


In [9]:
# get the bill id for the first row in the df and pass it to get_bill
# get_bill returns the bill info as a list with 1 dict inside it 
# (other bills may return more dicts, so check for that!)
# so, we have to open the list, open each dict, and use key 'name' to get the value
bill_id0 = df['Bill ID'][0]
bill_info0 = legis.get_bill(bill_id0)

In [10]:
print(bill_info0['sponsors'])

[{'people_id': 22862, 'person_hash': 'aqb8e73o', 'party_id': '2', 'state_id': 2, 'party': 'R', 'role_id': 1, 'role': 'Rep', 'name': 'Thomas McKay', 'first_name': 'Thomas', 'middle_name': 'W.', 'last_name': 'McKay', 'suffix': '', 'nickname': 'Tom', 'district': 'HD-024', 'ftm_eid': 49795890, 'votesmart_id': 194415, 'opensecrets_id': '', 'knowwho_pid': 739356, 'ballotpedia': 'Thomas_McKay', 'bioguide_id': '', 'sponsor_type_id': 1, 'sponsor_order': 1, 'committee_sponsor': 0, 'committee_id': 0, 'state_federal': 0}]


In [11]:
type(bill_info0['sponsors'])

list

In [12]:
type(bill_info0['sponsors'][0])

dict

In [13]:
bill_info0['sponsors'][0]['name']

'Thomas McKay'

In [14]:
# checking legialerts.com, it looks like df['Bill ID'][1] has multiple cosponsors, 
# so let's try getting the names from that one
# so, we have to open the list, open each dict, and use key 'name' to get the value
bill_id1 = df['Bill ID'][1]
bill_info1 = legis.get_bill(bill_id1)

In [15]:
# yup there's a lot of them!
print(bill_info1['sponsors'])

[{'people_id': 24239, 'person_hash': '5lotnoxe', 'party_id': '2', 'state_id': 3, 'party': 'R', 'role_id': 1, 'role': 'Rep', 'name': 'Rachel Jones', 'first_name': 'Rachel', 'middle_name': '', 'last_name': 'Jones', 'suffix': '', 'nickname': '', 'district': 'HD-017', 'ftm_eid': 55437931, 'votesmart_id': 205607, 'opensecrets_id': '', 'knowwho_pid': 0, 'ballotpedia': 'Rachel_Jones_(Arizona)', 'bioguide_id': '', 'sponsor_type_id': 1, 'sponsor_order': 1, 'committee_sponsor': 0, 'committee_id': 0, 'state_federal': 0}, {'people_id': 23000, 'person_hash': 'rv9jrvuk', 'party_id': '2', 'state_id': 3, 'party': 'R', 'role_id': 1, 'role': 'Rep', 'name': 'Lupe Diaz', 'first_name': 'Lupe', 'middle_name': '', 'last_name': 'Diaz', 'suffix': '', 'nickname': '', 'district': 'HD-014', 'ftm_eid': 0, 'votesmart_id': 200710, 'opensecrets_id': '', 'knowwho_pid': 791761, 'ballotpedia': 'Lupe_Diaz', 'bioguide_id': '', 'sponsor_type_id': 2, 'sponsor_order': 2, 'committee_sponsor': 0, 'committee_id': 0, 'state_fede

In [16]:
for sponsor in bill_info1['sponsors']:
    print(sponsor['name'])

Rachel Jones
Lupe Diaz
Liz Harris
Cory McGarr
Justine Wadsack


In [17]:
# let's see if this also works on the first bill
for sponsor in bill_info0['sponsors']:
    print(sponsor['name'])

Thomas McKay


---
# Get sponsors for all bills

In [18]:
dfbackup = df

In [19]:
df['Sponsors'] = ''

In [20]:
def get_sponsors(bill_id):
    try:
        bill_info = legis.get_bill(bill_id)
        sponsornames = []
        for sponsor in bill_info['sponsors']:
            name = sponsor['name']
            sponsornames.append(name)
        return sponsornames
    except:
        print(bill_id, "no sponsor found")        
        return None

In [21]:
get_sponsors(1644653)

['John Kavanagh']

In [ ]:
df['Sponsors'] = df.apply(lambda row: get_sponsors(row['Bill ID']), axis=1)

1685027 no sponsor found


In [ ]:
df.head()

In [ ]:
df.to_csv('anti-lgbtq-bills-sponsors.csv')
dfbackup = df

# Look at all sponsors
Let's see if pyttsx3 can pronounce these names...

In [ ]:
# turn the pandas series into a list
sponsors_unique = list(df['Sponsors'])

In [ ]:
# check it. it's a list of lists
sponsors_unique

In [ ]:
# flatten the nested lists — 
# for every sublist in toplist, for every item in that sublist, put that item into this [list]
flat_list = [item for sublist in sponsors_unique for item in sublist]

In [ ]:
# turn that into a set to remove repetitions of names
unique_sponsors = set(flat_list)

In [ ]:
# alphabetize them
unique_sponsors = sorted(unique_sponsors)

In [ ]:
unique_sponsors

# Speech to text

In [ ]:
pip install pyttsx3

In [ ]:
import pyttsx3
engine = pyttsx3.init()

In [ ]:
# default is SUPER FAST and kinda loud
engine.say("I will speak this text")
engine.runAndWait()

In [ ]:
""" RATE"""
rate = engine.getProperty('rate')   # getting details of current speaking rate

In [ ]:
print (rate)                        #printing current voice rate

In [ ]:
engine.setProperty('rate', 125)     # setting up new voice rate

In [ ]:
volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)

In [ ]:
print (volume)                          #printing current volume level

In [ ]:
engine.setProperty('volume',0.35)    # setting up volume level  between 0 and 1

In [ ]:
voices = engine.getProperty('voice')       #getting details of current voice

In [ ]:
print(voices)

In [ ]:
engine.say("I will speak this text")
engine.runAndWait()

In [ ]:
engine.setProperty('rate', 65)     # setting up new voice rate

In [ ]:
engine.setProperty('voice', voices[0].id)

In [ ]:
engine.say("I will speak this text")
engine.runAndWait()

# Curse

In [ ]:
# success!
for name in unique_sponsors:
    curse = f"A curse upon you, {name}. A curse upon your allies, and upon all who turn their faces from your works and close their eyes to your deeds. May we live untouched by the harm you seek to visit upon us. May we flourish as you wither. May we grow and change as your ways of life are forgotten and ground into dust by our dancing. May future generations look on you with disgust. May your name be scorned until it's forgotten. May you become powerless."
    engine.say(curse)
    engine.runAndWait()